In [9]:
# downloading all papers
import requests
import re

volume = 80
cont = requests.get('http://proceedings.mlr.press/v' + str(volume) + '/').text

paper_list = re.findall(
    r'(http://proceedings\.mlr\.press/v' + str(volume) + r'/.*?\.pdf)', cont)
paperlist = list(set(paper_list))

#for item in paperlist:
#    cont = requests.get(item).content
#    print('Download ' + item)
#    f = open(re.findall(r'http://proceedings\.mlr\.press/v' +
#                        str(volume) + r'/.*?/(.*?\.pdf)', item)[0], 'wb+')
#    f.write(cont)
#    f.close()

In [18]:
# put all text into single .txt file
import io, requests, string
from PyPDF2 import PdfFileReader

dump = open("dump.txt", "w")

def punctuation_extermination(s):
    o = str.maketrans('', '', string.punctuation)
    return str(s).translate(o)


for url in paperlist:
    r = requests.get(url)
    f = io.BytesIO(r.content)
    reader = PdfFileReader(f)
    contents = reader.getPage(0).extractText().split('\n')
    contentstring = str(contents)
    to_write = punctuation_extermination(contentstring)
    dump.write(to_write)

Superfluous whitespace found in object header b'1' b'0'
Superfluous whitespace found in object header b'2' b'0'
Superfluous whitespace found in object header b'3' b'0'
Superfluous whitespace found in object header b'16' b'0'
Superfluous whitespace found in object header b'15' b'0'
Superfluous whitespace found in object header b'4' b'0'
Superfluous whitespace found in object header b'5' b'0'
Superfluous whitespace found in object header b'6' b'0'
Superfluous whitespace found in object header b'7' b'0'
Superfluous whitespace found in object header b'8' b'0'
Superfluous whitespace found in object header b'9' b'0'
Superfluous whitespace found in object header b'10' b'0'
Superfluous whitespace found in object header b'11' b'0'
Superfluous whitespace found in object header b'12' b'0'
Superfluous whitespace found in object header b'13' b'0'
Superfluous whitespace found in object header b'14' b'0'
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexe

In [30]:
# top 10 most frequent words (w/o stopwords)
from nltk import FreqDist
from nltk.corpus import stopwords

corpus = []
with open('dump.txt','r') as file:
    corpus = [word for line in file for word in line.split()]

corpus_without_stop = [word for word in corpus if word.lower() not in stopwords.words("english")]

In [40]:
banned = ['et', 'al', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '2015', '2016', '2017', '2018', 'learning', 'models', 'algorithms', 'using']
# banning common sense / meaningless / plural words
corpus_without_stop = [word for word in corpus_without_stop if word.lower() not in banned]
freqdistr = FreqDist(corpus_without_stop)
top10words = [element[0] for element in freqdistr.most_common(10)]

top10words

['problem',
 'function',
 'algorithm',
 'training',
 'methods',
 'set',
 'gradient',
 'network',
 'number',
 'distribution']

In [65]:
# entropy
import math

megastring = ' '.join(corpus)
wrd = "regression" # random word
count_of_word = megastring.count(wrd)
total_length = len(set(megastring))
prob = count_of_word/total_length
log_prob = math.log(prob, 2)
entropy = prob * log_prob
entropy

0.2618514677902033

In [86]:
# generating paragraph using marginal distr

import pandas as pd
import numpy as np

df = pd.DataFrame()
df['words'] = [element[0] for element in freqdistr.most_common(1000000000000000)]
df['frequencies'] = [element[1] for element in freqdistr.most_common(1000000000000000)]
df['probabilities'] = df['frequencies']/len(corpus)
df['probabilities'] /= df['probabilities'].sum()
df['log_probabilities'] = np.log2(df['probabilities'])

p = np.random.choice(df['words'], 100, p=df['probabilities'])
p_string = ' '.join(x for x in p)

p_string

'AlvarezHamelin considered usually 2Michael Correspondence visualizing Adversarial reinforcement explicitly wl1vecx00 decomposition badly regret randomness depth Nash Beck thus athttpssitesgooglecomview 2007 R principle language iterative University methods problem comes study Abstract bias show even walk many Large allevi Castro guarantees Mnih write assessment obtain need utilization x0 solvers x151x00q solutions label object items Line deﬁne G4 us dataset allow spaces recently bestknown Sys cheaper constant make perturbed lognslashleftc2for v2Cjrvhjkvx00rhk2 Science samples fuv2Eis LM performances 3Varun Rx unknown separate Physics instances tuned dimension x12and recent distributed variables NNs ap Mnih Introduction expected ysuicaltechedu Correspondence numbers even Ixy China3Department OPE underlying regret networks'

In [68]:
# generate paragraph using n-gram
import string, random
from typing import List

def tokenize(text: str) -> List[str]:
    # tokenizer
    for punct in string.punctuation:
        text = text.replace(punct, ' '+punct+' ')
    t = text.split()
    return t

def get_ngrams(n: int, tokens: list) -> list:
    # make n-rams
    # tokens.append('<END>')
    tokens = (n-1)*['<START>']+tokens
    l = [(tuple([tokens[i-p-1] for p in reversed(range(n-1))]), tokens[i]) for i in range(n-1, len(tokens))]
    return l

class NgramModel(object):

    def __init__(self, n):
        self.n = n

        # dictionary that keeps list of candidate words given context
        self.context = {}

        # keeps track of how many times ngram has appeared in the text before
        self.ngram_counter = {}

    def update(self, sentence: str) -> None:
        # update lang model
        n = self.n
        ngrams = get_ngrams(n, tokenize(sentence))
        for ngram in ngrams:
            if ngram in self.ngram_counter:
                self.ngram_counter[ngram] += 1.0
            else:
                self.ngram_counter[ngram] = 1.0

            prev_words, target_word = ngram
            if prev_words in self.context:
                self.context[prev_words].append(target_word)
            else:
                self.context[prev_words] = [target_word]

    def prob(self, context, token):
        # conditional probability of word
        try:
            count_of_token = self.ngram_counter[(context, token)]
            count_of_context = float(len(self.context[context]))
            result = count_of_token / count_of_context

        except KeyError:
            result = 0.0
        return result

    def random_token(self, context):
        # randomly select next word
        r = random.random()
        map_to_probs = {}
        token_of_interest = self.context[context]
        for token in token_of_interest:
            map_to_probs[token] = self.prob(context, token)

        summ = 0
        for token in sorted(map_to_probs):
            summ += map_to_probs[token]
            if summ > r:
                return token

    def generate_text(self, token_count: int):
        # generator - set no. of words
        n = self.n
        context_queue = (n - 1) * ['<START>']
        result = []
        for _ in range(token_count):
            obj = self.random_token(tuple(context_queue))
            result.append(obj)
            if n > 1:
                context_queue.pop(0)
                if obj == '.':
                    context_queue = (n - 1) * ['<START>']
                else:
                    context_queue.append(obj)
        return ' '.join(result)


def create_ngram_model(n, path):
    m = NgramModel(n)
    with open(path, 'r') as f:
        text = f.read()
        text = text.split('.')
        for sentence in text:
            # add back the fullstop
            sentence += '.'
            m.update(sentence)
    return m

In [56]:
m = create_ngram_model(10, 'dump.txt')
# random.seed(2)
print(m.generate_text(100))

Decoupling GradientLike Learning Rules from Representations Philip S Thomas1Christoph Dann2Emma Brunskill3 Abstract In machine learning learning often corresponds to changing the parameters of a parameterized function A learning rule is an algorithm or math ematical expression that speciﬁes precisely how the parameters should be changed When creating a machine learning system we must make two decisions what representation should be used ie what parameterized function should be used and what learning rule should be used to search through the resulting set of representable func tions In this paper we focus on gradientlike learning rules wherein these two decisions are coupled
